In [ ]:
# Databricks notebook source
import mlflow
from mlflow.tracking import MlflowClient
import time
import os

print("🚀 UAT Staging Promotion Started...")

# =======================================================
# ✅ MLflow Registry (Unity Catalog)
# =======================================================
try:
    if "DATABRICKS_RUNTIME_VERSION" in os.environ:
        mlflow.set_registry_uri("databricks-uc")
        print("✅ Using Unity Catalog Registry")
    client = MlflowClient()
except Exception as e:
    print(f"❌ Failed to initialize MLflow client: {e}")
    raise e


# =======================================================
# ✅ FIXED MODEL NAME (aligned with training + registration)
# =======================================================
UC_CATALOG = "workspace"
UC_SCHEMA = "ml"
MODEL_NAME = f"{UC_CATALOG}.{UC_SCHEMA}.house_price_xgboost_uc"

STAGING_ALIAS = "Staging"
METRIC_KEY = "test_rmse"
TOL = 1e-6  # float tolerance


# =======================================================
# ✅ Helper: Wait until model version is READY
# =======================================================
def wait_until_ready(client, model_name, version, timeout=300):
    start = time.time()
    while time.time() - start < timeout:
        mv = client.get_model_version(model_name, version)
        status = mv.status
        print(f"⏳ Model v{version} status = {status}")

        if status == "READY":
            return True
        elif status == "FAILED_REGISTRATION":
            print("❌ Model registration failed.")
            return False
        
        time.sleep(5)

    print("⏰ Timeout: Model is still not READY")
    return False


# =======================================================
# ✅ Helper: get metric for a run
# =======================================================
def get_metric_from_run(client, run_id):
    try:
        run = client.get_run(run_id)
        return run.data.metrics.get(METRIC_KEY, None)
    except:
        return None


# =======================================================
# ✅ Step 1: Find Latest Model Version
# =======================================================
model_versions = client.search_model_versions(f"name='{MODEL_NAME}'")

if not model_versions:
    print(f"❌ No versions found for model: {MODEL_NAME}")
    raise SystemExit

latest_version = sorted(model_versions, key=lambda m: int(m.version), reverse=True)[0]
new_version = latest_version.version
new_run_id = latest_version.run_id
new_metric = get_metric_from_run(client, new_run_id)

print(f"\n✅ Latest Registered Model Version: v{new_version}")
print(f"✅ New Model RMSE: {new_metric}")


# =======================================================
# ✅ Step 2: Find existing Staging model (alias)
# =======================================================
try:
    staging_version = client.get_model_version_by_alias(MODEL_NAME, STAGING_ALIAS)
    old_version = staging_version.version
    old_run_id = staging_version.run_id
    old_metric = get_metric_from_run(client, old_run_id)

    print(f"\n📌 Current STAGING Version: v{old_version}")
    print(f"📌 Current STAGING RMSE: {old_metric}")

except Exception:
    print("\nℹ️ No current STAGING model. Will promote latest model.")
    staging_version = None
    old_metric = None


# =======================================================
# ✅ Step 3: Compare metrics (lower RMSE = better)
# =======================================================
promote = False

if staging_version is None:
    promote = True
    print("🟢 No model in staging → promoting latest model...")
else:
    if new_metric < old_metric - TOL:
        promote = True
        print("🟢 New model is BETTER → promoting to staging...")
    else:
        print("⛔ New model is NOT better. No promotion will occur.")


# =======================================================
# ✅ Step 4: Promote using alias = "Staging"
# =======================================================
if promote:
    print(f"\n⏳ Waiting for model v{new_version} to become READY...")
    if wait_until_ready(client, MODEL_NAME, new_version):

        client.set_registered_model_alias(
            name=MODEL_NAME,
            alias=STAGING_ALIAS,
            version=new_version
        )

        print(f"\n✅✅ SUCCESS: Model v{new_version} promoted to STAGING (alias).")
        print(f"⭐ New RMSE: {new_metric}")
    else:
        print("❌ Promotion failed due to model not becoming READY.")
else:
    print("\n✅ Staging remains unchanged.")
